In [9]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
import torch
from torchvision import datasets, transforms # 也是torch的工具，里面封装了更高级的功能
from torch.utils.data import DataLoader

In [10]:
# 训练集
train_dataset = datasets.MNIST(root='./', # 这里选择当前路径下载，意思就是说可以在当前路径MINST文件夹内找到数据，数据是封装好了的
                            train = True, # 下载训练集
                            transform = transforms.ToTensor(), #把数据变成基本的数据类型tensor
                            download = True) # 直接从封装好接口上载入

# 测试集
test_dataset = datasets.MNIST(root='./', # 这里选择当前路径下载
                            train = False, # 下载训练集
                            transform = transforms.ToTensor(), #把数据变成基本的数据类型tensor
                            download = True) # 直接从封装好接口上载入

train_dataset # 这个东西已经被封装成了60000个点，使用Data_loader拿出来用就好
len(train_dataset)


60000

In [11]:
# 批次大小，每次训练选择多少的数据
batch_size = 64 

# 装载训练集
# dataloader相当于创建了一个数据生成器
# 在这个里面数据就已经封装好了（train_loader和test_loader里面）
train_loader = DataLoader(dataset = train_dataset,  # 数据的来源
                          batch_size = batch_size,  # 批次的大小
                          shuffle = True) # 数据是否进行打乱

test_loader = DataLoader(dataset = test_dataset, # 来源于测试集
                         batch_size = batch_size,
                         shuffle = True)

train_loader

In [12]:
# 数据生成器使用方法——在循环中进行使用
for i,data in enumerate(train_loader): #enumerate是为了每一批次的数据增加一个索引i
    inputs, labels = data # data里面包含了要输入的量和输出的标签值
    print(inputs.shape)
    print(labels.shape)
    break
print(labels)
# 64个样本 1个通道（黑白） 28*28的像素
# 标签同样是64个，标签值代表了是哪个数据
len(train_loader) # 938*64 = 60032 刚好枚举完

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([5, 6, 6, 0, 5, 1, 5, 2, 4, 8, 2, 0, 7, 9, 8, 5, 8, 3, 8, 7, 9, 7, 7, 0,
        4, 7, 2, 9, 5, 3, 2, 7, 8, 7, 1, 7, 2, 2, 0, 9, 4, 4, 8, 2, 0, 5, 9, 7,
        1, 5, 7, 9, 8, 8, 3, 7, 8, 3, 0, 9, 8, 1, 1, 9])


938

# 对以下网络的一些解释
定义在网络结构里面的每一个背后都内含参数的
全连接层会对不同的样本进行分别处理
但是softmax是可以跨样本进行处理的（可能为了后续一些算法？）
如果dim=0，则会对每一列进行归一化，不符合我们要求，所以要dim=1，完成行归一化

# Dropout的相应用法
Dropout一般在隐藏层中使用，编写代码的时候写在网络结构层编写
在写网络结构时，可以使用封装代码
非封装：self.fc1 = nn.Linear(784,500)
封装版：self.layer1 = nn.Sequential(nn.Linear(784,500), nn.Dropout(p=0.5), nn.Tanh())
然后调用的时候直接用layer1即可

In [18]:
# 定义网络结构 Pytorch，一个网络结构（初始化） 一个网络计算（得到结果）
# 使用LSTM模型来进行图像的处理，图片本身是(64,1,28,28)，要把图片按行的顺序喂进去，喂28次
# LSTM各个参数含义
# input_size 每一个样本输入进去的序列长度（比如图片28行，则为28）
# hidden_size LSTM每一层的模块数，这里取64，则序列里每一个xn输出的会有64个hn
# num_layers 层数，一般1层即可
# batch_first 默认的input为（seq_len, batch, feature），True后为（batch, seq_len, feature），是三维数据
# 模型的大小一定要匹配！
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(
            input_size = 28,
            hidden_size = 64,
            num_layers = 1,
            batch_first = True
        )
        self.out = torch.nn.Linear(in_features=64, out_features=10) # 我只取sequence最后一个对应的输出，进行全连接
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        # x 输入的时候维度是（64,1,28,28）, 现在我要变成三维数据来处理
        # output：[batch, seq_len, hidden_size]包含每个序列输出的结果
        # h_n：[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        # c_n：[num_layers, batch, hidden_size]
        x = x.reshape(-1, 28, 28) # 这里就变成了（64，28，28），可以输入LSTM中
        output, (h_n, c_n) = self.lstm(x)
        output_in_last_timestep = h_n[-1,:,:]
        x = self.out(output_in_last_timestep)
        x = self.softmax(x)
        return x
    

In [19]:
# 定义模型（对象化）
model = LSTM() #对象化也是一个方法
# 定义代价函数
# 这是一个类，你需要实例化这个类！
mse_loss = nn.CrossEntropyLoss() #交叉熵损失
# 定义优化器
# 要传入模型参数和学习率
LR = 0.001
optimizer = optim.Adam(model.parameters(), LR)

In [20]:
# 进行模型的训练和测试方法——一边训练一边测试模型效果
def train():
    # 在dropout里面需要额外加上model.train()
    model.train() # 代表是训练状态，此时模型的dropout是起作用的
    for i,data in enumerate(train_loader): # 对迭代器进行分析
        # 循环一次，就会获得一次数据的批次和标签，这很重要！
        inputs, labels = data
        # 获得模型预测结果, 这个结果是[64, 10]的一个矩阵，64是批次，10是概率值
        out = model(inputs)
        
        # 用交叉熵损失，用交叉损失时，函数会内部自动转换成独热编码
        # 其中，out是（64，10）数组，labels是64的一位数组
        # 根据这个函数的用法，两者不需要对齐，只要保证第一个数都是minibatch值即可
        loss = mse_loss(out, labels)
        
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

def test():
    model.eval()
    # 测试集准确率
    correct = 0
    for i,data in enumerate(test_loader): # 每一次都是在对一个训练好的模型进行评估，把所有的测试集跑了一边
        inputs, labels = data
        out = model(inputs)
        # torch.max计算完后得到两个值，获得最大值和最大值所在的位置，dim=1代表对每一行进行操作
        # 这里会对64行每一行都进行操作的
        _, predicted = torch.max(out, 1)
        # 正确个数
        correct += (predicted == labels).sum() # 这里面的true和false会一个一个对比
    print('Test acc:{0}'.format(correct.item()/len(test_dataset)))

    # 训练集准确率
    correct = 0
    for i,data in enumerate(train_loader): # 每一次都是在对一个训练好的模型进行评估，把所有的测试集跑了一边
        inputs, labels = data
        out = model(inputs)
        # torch.max计算完后得到两个值，获得最大值和最大值所在的位置，dim=1代表对每一行进行操作
        # 这里会对64行每一行都进行操作的
        _, predicted = torch.max(out, 1)
        # 正确个数
        correct += (predicted == labels).sum() # 这里面的true和false会一个一个对比
    print('Train acc:{0}'.format(correct.item()/len(train_dataset)))

# Dropout 使用场景
并不一定让准确度变高，在模型比较复杂的情况下，应当使用，防止过拟合

In [21]:
# 循环运行
# 获得了更快的收敛速度
for epoch in range(20):
    print('epoch:', epoch)
    train()
    test()

epoch: 0


KeyboardInterrupt: 

# 这里是一个常见的框架
数据集的载入
定义批次大小和分割数据集（接下去在循环中对这个数据进行训练）
定义网络结构 和 网络运算 def __init__(self) 和 def forward(self, x)
对象化模型 代价函数 优化器
写网络如何训练测试——用两个模块封装好
训练：输出结果 求损失 梯度归零 更新参数
测试：输出结果

一个大循环